In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [76]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [77]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар и сравните с качеством на семинаре.

В фильме кин-дза-дза был такой термин: "Ку - все остальные слова". В нашем случае - все остальные товары.

Нужен он для того, чтобы упростив данные по товарам, не затронуть структуру пользователей, потеряв тех кто не покупал ни одного популярного товара.

Поскольку из финальных предсказаний выкидывается (он является абстрактным, и не может быть продан), вероятность быть предсказанными для всех непопулярных товаров обнуляется, перераспределяясь между популярными. Однако как именно - зависит от того, какую долю малопродаваемых товаров мы откинули. Предельный случай, если выброшен 1 непопулярный товар - остальные будут предсказаны без изменений. С уменьшением кол-ва нашего "ТОПа", суммарная популярность отброшенных товаров начинает расти, и, со временем, те товары, которые будут выше в рейтинге РС, чем 999999, с большой долей вероятности останутся на своих местах. Те, что ниже - сдвинутся на позицию. При "выпиливании" достаточно большого числа товаров, 999999 станет "самым рекоммендуемым". Резюмируя, после замены части товаров единым, собирательным, РС "сосредоточится" на предсказании товаров из оставленного ТОПа.

Про удаление - не конкретизировано, на каком этапе. Из финальной выдачи он должен быть удален. На этапе подготовки данных, выбрасывать его чревато. Можем нарушить структуру пользователей. Но если данных достаточно, и очень хочется, можно сначала очистить датасет пользователей, для которых 999999 - единственная покупка, а потом удалить товар.

Сам подход имеет существенный минус, который необходимо учитывать (сделать отдельную РС, как вариант). В отбрасываемые товары очевидно, попадут многие "новинки". Эти товары, вероятно, не вошли в ТОП не по причине малого спроса, а из-за того, что из-за малого времени не успели набрать продажи. Получается "порочный круг" - новинки не предлагаются, поэтому продаются хуже чем могли бы, поэтому меньше шансов когда-нибудь попасть в ТОП продаваемых. Для бизнеса интерес прямо противоположный - предложить новинки и как можно скорее собрать статистику по ним. Логично сделать отдельную РС "новинки" для этого.


In [78]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)  # n_sold - число продаж
popularity.head()


,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [79]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

In [80]:
data_train = data_train.loc[data_train['item_id'].isin(top_5000), :]

In [81]:
# Перевем данные в вид матрицы интеракций
user_item_matrix = pd.pivot_table(data_train, index='user_id', columns='item_id', values='quantity',
                                  aggfunc='count', fill_value=0)

user_item_matrix[user_item_matrix > 0] = 1  # так как в итоге хотим предсказать
user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [82]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [83]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 2.91 s, sys: 187 ms, total: 3.09 s
Wall time: 1.46 s


In [84]:

recs

[(3407, 70158.0),
 (2148, 57211.0),
 (3586, 27050.0),
 (3946, 19897.0),
 (2307, 18127.0)]

In [85]:
result = pd.read_csv('data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846312, 9823000, 900243, 10309250, 588311]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13511611, 1021529, 928537, 956887, 908361]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [86]:

%%time

result['itemitem_only_5000'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

CPU times: user 14.1 ms, sys: 3.96 ms, total: 18 ms
Wall time: 17 ms


In [87]:
features = ['user_id', 'itemitem', 'itemitem_only_5000']
result[features].head()

,user_id,itemitem,itemitem_only_5000
0,1,"[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1098066, 1127831, 995242]"
1,3,"[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 1127831, 995242]"
2,6,"[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1098066, 1127831, 995242]"
3,7,"[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1098066, 1127831, 995242]"
4,8,"[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 1098066, 1127831, 995242]"


Видно, что без использования товара 999999, позиции ровно такие же, но смещены на один индекс. 

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [88]:
log_items_weights = data.groupby(by='item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value':'weight'})

log_items_weights['weight'] += 1 # Чтобы избежать нулевых весов
log_items_weights['weight'] = np.log(log_items_weights['weight'])
log_items_weights['weight'] /= log_items_weights['weight'].sum()
log_items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000006
min,2.567100e+04,0.000000
25%,9.665830e+05,0.000006
50%,1.448516e+06,0.000010
75%,9.553042e+06,0.000016
max,1.802456e+07,0.000053


In [89]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(log_items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=log_items_weights['weight'])
    
    return recs.tolist()

In [90]:
%%time

%%time
result['weighted_log_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_items_weights, n=5))
result.head(2)

CPU times: user 3.41 s, sys: 7.95 ms, total: 3.42 s
Wall time: 3.47 s
CPU times: user 3.41 s, sys: 7.96 ms, total: 3.42 s
Wall time: 3.47 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_only_5000,weighted_log_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846312, 9823000, 900243, 10309250, 588311]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 981760, 1098066, 1127831, 995242]","[842362, 4414500, 9419323, 9296859, 824151]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13511611, 1021529, 928537, 956887, 908361]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]","[6704442, 965982, 869577, 8181020, 885294]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [91]:
# result = pd.read_csv('data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_only_5000,weighted_log_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846312, 9823000, 900243, 10309250, 588311]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 981760, 1098066, 1127831, 995242]","[842362, 4414500, 9419323, 9296859, 824151]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13511611, 1021529, 928537, 956887, 908361]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]","[6704442, 965982, 869577, 8181020, 885294]"


При попытке посчитать метрику получал ошибку IndexError: too many indices for array. Поколупавшись часик, выяснил что данные "приезжают" в виде строки, и np.array() не преобразовывает их корректно. Написал костыль для явного преобразования в список, но предполагаю что должно быть более изящное решение.

In [92]:
def precision_at_k(recommended_list, bought_list, k=5):

    if isinstance(bought_list, str):
        str_list = bought_list[1:-1].split(', ')
        bought_list = [int(item) for item in str_list]
    if isinstance(recommended_list, str):
        str_list = recommended_list[1:-1].split(', ')
        recommended_list = [int(item) for item in str_list]
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    

    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)  
    

    
    precision = flags.sum() / len(recommended_list)    
    
    return precision

In [93]:

pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

,model name,precision@k
0,random_recommendation,0.00059
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,itemitem_only_5000,0.14574
7,weighted_log_random_recommendation,0.00127


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


Отвечая на последний вопрос нулевого задания, метрика при удалении товара 999999 выросла, что не удивительно - наш квази-товар никто не покупал, а в предсказаниях он фигурировал как самый продаваемый. 

In [94]:
result.apply(lambda row: precision_at_k(row['itemitem_only_5000'], row['actual'], k=4), axis=1).mean()

0.14409892262487758

In [95]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'][1:], k=4), axis=1).mean()

0.14666993143976495

Посчитал, выбросив товар 999999 и последнее значение itemitem_only_5000, чтобы уровнять количество, и модель "с товаром" показала лучший результат. Таким образом, бейзлайн itemitem был улучшен. По хорошему, нам нужно список из 6 предсказаний, если хотим оставить 5 товаров.

In [96]:
def random_top_recommendation(data_train, top=20, n=5):
    """Случайные рекоммендации среди популярных товаров"""
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    items = np.array(popularity.sort_values('quantity', ascending=False).head(top).item_id.tolist())    
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [97]:
%%time
popular_recs = random_top_recommendation(data_train, n=5)
result['random_top_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 48.7 ms, sys: 11 µs, total: 48.7 ms
Wall time: 47.6 ms


In [98]:
# for k in range(1, 20):
#     model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

#     model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
#               show_progress=True)

#     recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
#                             user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
#                             N=5, # кол-во рекомендаций 
#                             filter_already_liked_items=False, 
#                             filter_items=None, 
#                             recalculate_user=True)
    
#     result[f'itemitem_K={k}'] = result['user_id'].\
#     apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

При достижении K=8 метрика достигает максимального значения, и дальше не меняется. Возьмем K=9, чтобы исключить случайные "просадки".

In [99]:
%%time

model = ItemItemRecommender(K=9, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 2.85 s, sys: 247 ms, total: 3.1 s
Wall time: 1.37 s


In [100]:
%%time

result['itemitem_K=9'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

CPU times: user 5.52 ms, sys: 0 ns, total: 5.52 ms
Wall time: 5.43 ms


In [101]:
pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

,model name,precision@k
0,random_recommendation,0.00059
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,itemitem_only_5000,0.14574
7,weighted_log_random_recommendation,0.00127
8,random_top_recommendation,0.00059
9,itemitem_K=9,0.16357


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 

Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

$\mu$ - средняя оценка по всей таблице

$\bar{r_v}$ - средний рейтинг товаров, которые оценил пользователь Для одних 7 звезд - очень плохой товар, для других - очень хороший. $\bar{r_{v,i}} - \bar{r_v}$ позволяет определить оценку, в сравнении с обычной оценкой пользователя.

$N_i(u)$ - массив пользователей N которые оценили товар i, который оценил пользователь u

$u$ - пользователь

$v$ - пользователь из множества $N_i(u)$

$i$ - товар

$r_{u,i}$ - оценка которая мы предсказываем паре



In [102]:
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

In [104]:
mu = user_item_matrix.mean().mean() # находим среднюю оценку всех товаров
ru_overline = user_item_matrix.mean(axis=1)# находим среднюю оценку данного пользователя

In [105]:
cosine_matrix = (user_item_matrix.T - ru_overline.values - mu).T

In [106]:
# user_item_matrix

In [107]:
cosine_matrix.astype(np.float)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655,...,-0.27655,-0.27655,-0.27655,-0.27655,0.72345,-0.27655,-0.27655,-0.27655,-0.27655,-0.27655
2,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,...,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655,-0.19655
3,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,...,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775,-0.25775
4,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,...,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115,-0.15115
5,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,...,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275,-0.13275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,...,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595,-0.31595
2497,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,...,-0.37815,0.62185,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815,-0.37815
2498,1.79105,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,...,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895,-0.20895


In [108]:
%%time

model = ItemItemRecommender(K=9, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(cosine_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(cosine_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 4min 6s, sys: 1.84 s, total: 4min 8s
Wall time: 1min 23s


In [109]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 6534178, 995242, 1029743, 981760]

In [110]:
%%time

result['itemitem_K=9_cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

CPU times: user 15.9 ms, sys: 3.97 ms, total: 19.9 ms
Wall time: 18.2 ms


In [111]:
pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

,model name,precision@k
0,random_recommendation,0.00059
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,itemitem_only_5000,0.14574
7,weighted_log_random_recommendation,0.00127
8,random_top_recommendation,0.00059
9,itemitem_K=9,0.16357
